In [34]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import math
from external.metric import ShortLexBasisBladeOrder, construct_gmt, gmt_element
from external.mvsilu import MVSiLU
#from external.mvlayernorm import MVLayerNorm

In [35]:
metric = [1,1]
d = len(metric)
N = 1024 # sample size
# sample two vectors u,v per sample
x = torch.randn(N,2,d)
y = torch.randint(1,3,(N,)) # sample 1 or 2 for y, a categorical variable

$$f(u, v, y):= \begin{cases}\cos(\lVert u \rVert) &y=1 \\
\frac{\langle u, v \rangle ^3}{10} \qquad &y=2
\end{cases}$$

In [36]:
# compute the target values
f = torch.zeros(N)
metric_t = torch.tensor(metric,dtype=torch.float) # convert metric to tensor type
f[y==1] = torch.cos(torch.einsum("bi,i,bi->b",x[y==1][:,0],metric_t,x[y==1][:,0]).abs().sqrt()) # compute u norm via u^TMu
f[y==2] = 1/10 * torch.einsum("bi,i,bi->b",x[y==2][:,0],metric_t,x[y==2][:,1])**3 

In [37]:
class CliffordAlgebra(nn.Module):
    def __init__(self, metric):
        super().__init__()
        # include metric as a tensor to be part of state but won't be updated by optimizer
        self.register_buffer("metric", torch.as_tensor(metric)) # signature of symmetric bilinear form
        self.vspace_dim = len(metric) # vector space dimension = number of vector basis
        self.bbo = ShortLexBasisBladeOrder(self.vspace_dim) # get ordered algebra basis (external)
        self.algebra_dim = len(self.bbo.grades) # algebra dimension
        # construct cayley table (external)
        cayley = (
            construct_gmt(
                self.bbo.index_to_bitmap, self.bbo.bitmap_to_index, self.metric
            )
            .to_dense()
            .to(torch.get_default_dtype())
        )
        self.register_buffer("cayley", cayley)
        self.grades = self.bbo.grades.unique() # all grade numbers (e.g. [0,1,2,3] for n = 3) as a tensor
        self.register_buffer("subspaces",torch.as_tensor([math.comb(self.vspace_dim, grade) for grade in self.grades])) # [1 3 3 1] for n = 3
        self.n_subspaces = len(self.subspaces)
        self.grade2slice = self._grade2slice()
        

    def geometric_product(self, x, y, ):
        cayley = self.cayley
        return torch.einsum("...i, ikj, ...j -> k", x, cayley, y)
    
    def embed_grade(self, tensor: torch.Tensor, grade: int) -> torch.Tensor:
        mv = torch.zeros(*tensor.shape[:-1], 2**self.vspace_dim, device=tensor.device)
        s = self.grade2slice[grade]
        mv[..., s] = tensor 
        return mv

    
    def _grade2slice(self):
        grade2slice = list()
        #convert subspaces data (a list of counts of subspaces with increasing dimension) to tensor e.g. [1 3 3 1] for n=3
        subspaces = torch.as_tensor(self.subspaces)
        for grade in self.grades:
            index_start = subspaces[:grade].sum()
            index_end = index_start + math.comb(self.vspace_dim, grade)
            grade2slice.append(slice(index_start, index_end))
        return grade2slice

    
    def geometric_product_paths(self):
        # dim+1 since we have scalars as 0-dim
        gp_paths = torch.zeros((self.vspace_dim + 1, self.vspace_dim + 1, self.vspace_dim + 1), dtype=bool)
        # 
        for i in range(self.vspace_dim + 1):
            for j in range(self.vspace_dim + 1):
                for k in range(self.vspace_dim + 1):
                    s_i = self.grade_to_slice[i]
                    s_j = self.grade_to_slice[j]
                    s_k = self.grade_to_slice[k]
                    
                    # m is a 3D tensor, capturing whether basis of subspaces of two grades give any basis of the third grade
                    # e.g. i = 2, j = 2, k = 1, do basis (e12, e23, e13) and (e12, e23, e13) give any 1-blade basis? No. So
                    # gp_paths(2,2,1) = False/0
                    m = self.cayley[s_i, s_j, s_k]
                    gp_paths[i, j, k] = (m != 0).any()

        return gp_paths

In [38]:
class MVLinear(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()

        self.algebra = algebra
        self.in_features = in_features
        self.out_features = out_features

        self.weights = nn.Parameter(torch.empty(out_features,in_features,algebra.n_subspaces))
        self.normalize_parameters()

    def normalize_parameters(self):
        torch.nn.init.normal_(self.weights, std = 1/ math.sqrt(self.in_features))

    # we have l number of input channels (batch size)
    def forward(self, input):
        weights = self.weights.repeat_interleave(self.algebra.subspaces,dim=-1) #repeat along the last dimension (n_subspaces) of weights
        return torch.einsum("bm...i,nmi -> bn...i", input, weights) #output for each batch b, output feature n, and subspace i, ... represents shape of a single datapoint
        

In [ ]:
class FullyConnectedGeometricProductLayer(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()
        self.algebra = algebra
        self.in_features = in_features
        self.out_features = out_features

        self.linear_right = MVLinear(algebra, in_features, in_features)
        self.linear_left = MVLinear(algebra,in_features,out_features)

        self.normalization = nn.Identity()

        self.product_paths = algebra.geometric_product_paths
        self.weights = nn.Parameter(torch.empty(out_features,in_features,self.product_paths.sum()))

        self.reset_parameters()

    def reset_parameters(self):
        nn.init.normal(self.weights, 
                       std = 2/math.sqrt(self.in_features * (self.algebra.vspace_dim + 1)) #for ReLU, we want total input features, have one for each grade)
                       )
        
    def _get_weights(self):
        weights = torch.zeros(
            self.out_features, self.in_features, *self.product_paths.size(),dtype=self.weights.dtype , device=self.weights.device
        )
        subspaces = self.algebra.subspaces
        weights[:,:,self.product_paths] = self.weights
        weights_repeated = weights.repeat_interleave(subspaces, dim = -3).repeat_interleave(subspaces, dim = -2). repeat_interleave(subspaces, dim = -1)
        return self.algebra.cayley * weights_repeated
    
    def forward(self,input):
        input_right = self.linear_right(input)
        input_right = self.normalization(input_right)
        weights = self._get_weights()
        return (self.linear_left(input) + torch.einsum("bni, mnijk, bnk -> bmj",input, weights, input_right))/ math.sqrt(2) # eq 15 and 14, i + k = j, more normalization


In [39]:
class CGEBlock(nn.Module):
    def __init__(self, algebra, in_features, out_features):
        super().__init__()
        self.layers = nn.Sequential(
            MVLinear(algebra,in_features,out_features)#,
            #MVSiLU(algebra,out_features),
            #MVLayerNorm(algebra,out_features)
        )
    def forward(self,input):
        return self.layers(input)

In [40]:
class CGEMLP(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_layers = 2):
        super().__init__()
        layers = []
        for i in range(n_layers-1):
            layers.append(CGEBlock(algebra,in_features,hidden_features))
            in_features = hidden_features
        layers.append(CGEBlock(algebra,hidden_features,out_features))
        self.layers = nn.Sequential(*layers)
    def forward(self,input):
        return self.layers(input)

In [41]:
ca = CliffordAlgebra(metric)
x_cl = ca.embed_grade(x,1)
print(x_cl.shape)
print(x_cl[0])

torch.Size([1024, 2, 4])
tensor([[ 0.0000,  0.3261,  0.0647,  0.0000],
        [ 0.0000, -0.7778,  0.2760,  0.0000]])


In [42]:
y_oh = F.one_hot(y - 1, 2)
y_oh

tensor([[0, 1],
        [0, 1],
        [0, 1],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]])

In [43]:
y_cl = ca.embed_grade(y_oh[..., None], 0)
print(y_cl.shape)
print(y_cl[0])

torch.Size([1024, 2, 4])
tensor([[0., 0., 0., 0.],
        [1., 0., 0., 0.]])


In [44]:
input_cl = torch.cat([x_cl, y_cl], dim=1)

In [45]:
model = CGEMLP(ca,4,32,1)

In [47]:
print(f"The model has {sum(p.numel() for p in model.parameters())} parameters.\n")
adam = optim.Adam(model.parameters())

for i in range(256):

    output = model(input_cl)
    loss = F.mse_loss(output.squeeze(-1), f)

    adam.zero_grad()
    loss.backward()
    adam.step()

    if i % 4 == 0:
        print(f"Step: {i}. Loss: {loss.item():.2f}")

The model has 480 parameters.



C:\Users\bryan\AppData\Local\Temp\ipykernel_24000\524895709.py:7: UserWarning: Using a target size (torch.Size([1024])) that is different to the input size (torch.Size([1024, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(output.squeeze(-2), f)


RuntimeError: The size of tensor a (4) must match the size of tensor b (1024) at non-singleton dimension 1

: 